In [1]:
from newscatcher import Newscatcher, describe_url, urls
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import numpy as np
import math
import time
import calendar
import json
import sqlite3

In [2]:
base_urls = []

with open('base_urls.txt', 'r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        base_urls.append(lines[i].strip())

In [3]:
# 2021. 4. 11 기준 사용 가능한 소스 1514개 (wsj, imf, nyt 등등)인데, 잘 알려진 신문(뉴스)으로 제한함 
# (밑으로 가도 잘 모르는 신기한 언론사만 추가될 뿐이라고 생각함) + (다운받을 text가 너무 많아져서 시간 상의 문제가 발생)
len(base_urls)

30

In [4]:
# [source, topic, title, publish_date, link, text] --> 이렇게 해서 link를 뉴스 기사 url로 보고 newspaper3k 패키지로 내용 가져올 것
# link까지가 newscatcher, text는 newspaper3k로 가져옴

news_base_data = []
number = 0

for base_url in base_urls:
    
    number += 1
    print(number)
    
    # base_url의 topic들
    topics = describe_url(base_url)['topics']
    
    # topic들 각각에 대하여
    try:
        for topic in topics:
            nc = Newscatcher(base_url, topic = topic)
            results = nc.get_news()
            articles = results['articles']

            # base_url - topic pair에 맞는 기사 각각에 대하여
            for article in articles:
                title = article['title']
                pub_date = article['published']
                link = article['link']

                temp = []

                temp.append(base_url)
                temp.append(topic)
                temp.append(title)
                temp.append(pub_date)
                temp.append(link)

                news_base_data.append(temp)

    except:
        print(base_url, topic, title)

1
2
3
4
cnn.com world $300K Refi in this Economy? 2.10% APR (15 yr)
5
6

No results found check internet connection or query parameters

washingtonpost.com tech Jailed Navalny to be moved to a hospital in another prison
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
prweb.com sport PXG LPGA Tour Pro Lydia Ko Delivers Knockout Performance at the Lotte...
27
28
29
30


In [5]:
news_base_data

[['yahoo.com',
  'tech',
  "Amazon's Fire HD 8 and Blink Mini camera bundle is $75 for today only",
  '2021-04-19T10:15:41Z',
  'https://finance.yahoo.com/news/amazon-fire-hd-tablet-blink-mini-camera-bundle-101541834.html'],
 ['yahoo.com',
  'tech',
  "Sony's FlavorGraph uses AI to predict which ingredients will pair together",
  '2021-04-19T09:29:35Z',
  'https://finance.yahoo.com/news/sonys-flavor-graph-uses-ai-to-predict-how-ingredients-will-pair-together-092935932.html'],
 ['yahoo.com',
  'tech',
  "Volvo to supply cars for Didi's global autonomous driving fleets",
  '2021-04-19T08:21:37Z',
  'https://finance.yahoo.com/news/volvo-supply-chinese-ride-hailing-082137015.html'],
 ['yahoo.com',
  'tech',
  'Andela begins global expansion in 37 countries months after going remote across Africa',
  '2021-04-19T07:57:09Z',
  'https://finance.yahoo.com/news/andela-begins-global-expansion-37-075709662.html'],
 ['yahoo.com',
  'tech',
  'New York caps the cost of broadband for low-income fami

In [6]:
len(news_base_data)

4863

In [7]:
df_news_base_data = pd.DataFrame(news_base_data, columns=['source','topic','title','publish_date','link'])

In [8]:
df_news_base_data

,source,topic,title,publish_date,link
0,yahoo.com,tech,Amazon's Fire HD 8 and Blink Mini camera bundl...,2021-04-19T10:15:41Z,https://finance.yahoo.com/news/amazon-fire-hd-...
1,yahoo.com,tech,Sony's FlavorGraph uses AI to predict which in...,2021-04-19T09:29:35Z,https://finance.yahoo.com/news/sonys-flavor-gr...
2,yahoo.com,tech,Volvo to supply cars for Didi's global autonom...,2021-04-19T08:21:37Z,https://finance.yahoo.com/news/volvo-supply-ch...
3,yahoo.com,tech,Andela begins global expansion in 37 countries...,2021-04-19T07:57:09Z,https://finance.yahoo.com/news/andela-begins-g...
4,yahoo.com,tech,New York caps the cost of broadband for low-in...,2021-04-19T07:29:23Z,https://finance.yahoo.com/news/new-york-caps-t...
...,...,...,...,...,...
4858,macrobusiness.com.au,economics,"Chinese data misses big, economy slows fast","Fri, 16 Apr 2021 03:00:56 +0000",https://www.macrobusiness.com.au/2021/04/chine...
4859,macrobusiness.com.au,economics,Will Bitcoin destroy the US dollar?,"Fri, 16 Apr 2021 02:30:55 +0000",https://www.macrobusiness.com.au/2021/04/will-...
4860,macrobusiness.com.au,economics,Easter holidays dampen auction market,"Fri, 16 Apr 2021 02:00:43 +0000",https://www.macrobusiness.com.au/2021/04/easte...
4861,macrobusiness.com.au,economics,China’s little finaincal crisis rolls on,"Fri, 16 Apr 2021 02:00:30 +0000",https://www.macrobusiness.com.au/2021/04/china...


In [9]:
df_news_base_data = df_news_base_data.drop_duplicates()
df_news_base_data = df_news_base_data.reset_index(drop=True)
df_news_base_data

,source,topic,title,publish_date,link
0,yahoo.com,tech,Amazon's Fire HD 8 and Blink Mini camera bundl...,2021-04-19T10:15:41Z,https://finance.yahoo.com/news/amazon-fire-hd-...
1,yahoo.com,tech,Sony's FlavorGraph uses AI to predict which in...,2021-04-19T09:29:35Z,https://finance.yahoo.com/news/sonys-flavor-gr...
2,yahoo.com,tech,Volvo to supply cars for Didi's global autonom...,2021-04-19T08:21:37Z,https://finance.yahoo.com/news/volvo-supply-ch...
3,yahoo.com,tech,Andela begins global expansion in 37 countries...,2021-04-19T07:57:09Z,https://finance.yahoo.com/news/andela-begins-g...
4,yahoo.com,tech,New York caps the cost of broadband for low-in...,2021-04-19T07:29:23Z,https://finance.yahoo.com/news/new-york-caps-t...
...,...,...,...,...,...
4858,macrobusiness.com.au,economics,"Chinese data misses big, economy slows fast","Fri, 16 Apr 2021 03:00:56 +0000",https://www.macrobusiness.com.au/2021/04/chine...
4859,macrobusiness.com.au,economics,Will Bitcoin destroy the US dollar?,"Fri, 16 Apr 2021 02:30:55 +0000",https://www.macrobusiness.com.au/2021/04/will-...
4860,macrobusiness.com.au,economics,Easter holidays dampen auction market,"Fri, 16 Apr 2021 02:00:43 +0000",https://www.macrobusiness.com.au/2021/04/easte...
4861,macrobusiness.com.au,economics,China’s little finaincal crisis rolls on,"Fri, 16 Apr 2021 02:00:30 +0000",https://www.macrobusiness.com.au/2021/04/china...


In [10]:
# topic은 news, business, finance, politics, economics, world (시장에 영향을 줄 수 있다고 생각되는 topic)
is_news = df_news_base_data['topic']=='news'
is_business = df_news_base_data['topic']=='business'
is_finance = df_news_base_data['topic']=='finance'
is_politics = df_news_base_data['topic']=='politics'
is_economics = df_news_base_data['topic']=='economics'
is_world = df_news_base_data['topic']=='world'

df_news_base_data = df_news_base_data[is_news|is_business|is_finance|is_politics|is_economics|is_world].reset_index(drop=True)

In [11]:
df_news_base_data

,source,topic,title,publish_date,link
0,yahoo.com,news,Police: Suspect on the run in fatal shooting o...,2021-04-18T18:15:49Z,https://news.yahoo.com/ems-3-adults-fatally-sh...
1,yahoo.com,news,'Highly unlikely' Chauvin trial ends in 'all-o...,2021-04-18T18:19:00Z,https://news.yahoo.com/highly-unlikely-chauvin...
2,yahoo.com,news,Israeli intel agencies believe Vienna talks wi...,2021-04-18T19:06:19Z,https://news.yahoo.com/israeli-intel-agencies-...
3,yahoo.com,news,DeSantis Hints at Political Consequences for C...,2021-04-18T15:58:42Z,https://news.yahoo.com/desantis-hints-politica...
4,yahoo.com,news,Suspect apprehended in fatal shooting at Wisco...,2021-04-18T10:00:55Z,https://news.yahoo.com/police-3-dead-2-wounded...
...,...,...,...,...,...
2633,macrobusiness.com.au,economics,"Chinese data misses big, economy slows fast","Fri, 16 Apr 2021 03:00:56 +0000",https://www.macrobusiness.com.au/2021/04/chine...
2634,macrobusiness.com.au,economics,Will Bitcoin destroy the US dollar?,"Fri, 16 Apr 2021 02:30:55 +0000",https://www.macrobusiness.com.au/2021/04/will-...
2635,macrobusiness.com.au,economics,Easter holidays dampen auction market,"Fri, 16 Apr 2021 02:00:43 +0000",https://www.macrobusiness.com.au/2021/04/easte...
2636,macrobusiness.com.au,economics,China’s little finaincal crisis rolls on,"Fri, 16 Apr 2021 02:00:30 +0000",https://www.macrobusiness.com.au/2021/04/china...


In [12]:
# df_news_base_data하고 news_urls 테이블에 있는 것의 차집합을 구한 뒤 그 차집합을 news_urls 테이블에 업데이트함
con = sqlite3.connect('news_urls.db')
df_news_url_db = pd.read_sql('select * from news_urls', con=con)
con.close()

In [13]:
# 원래 있던 db와 비교해야 하니 원래 db를 가져옵니다.
df_news_url_db = df_news_url_db.loc[:, ['source','topic','title','publish_date','link']]
df_news_url_db

,source,topic,title,publish_date,link
0,yahoo.com,news,Iran vows revenge against Israel for strike on...,2021-04-12T10:11:31Z,https://news.yahoo.com/iran-vows-revenge-again...
1,yahoo.com,news,"Georgia sheriff: 3 officers wounded, 1 suspect...",2021-04-12T11:21:10Z,https://news.yahoo.com/sheriffs-official-georg...
2,yahoo.com,news,"AP PHOTOS: From Moscow to Pacific, Russia glor...",2021-04-12T07:35:18Z,https://news.yahoo.com/ap-photos-moscow-pacifi...
3,yahoo.com,news,Biden hasn't actually lifted Trump's historica...,2021-04-12T10:21:00Z,https://news.yahoo.com/biden-hasnt-actually-li...
4,yahoo.com,news,"Fiery crash kills driver, sends passenger to S...",2021-04-12T13:08:06Z,https://news.yahoo.com/fiery-crash-kills-drive...
...,...,...,...,...,...
8382,bostonglobe.com,news,Bridgette Mitchell named Northeastern women’s ...,"Sat, 17 Apr 2021 19:10:29 +0000",https://www.bostonglobe.com/2021/04/17/sports/...
8383,bostonglobe.com,news,State bans biomass power plants near environme...,"Sat, 17 Apr 2021 18:53:35 +0000",https://www.bostonglobe.com/2021/04/16/science...
8384,bostonglobe.com,news,Taking stock of the Bruins after the trade dea...,"Sat, 17 Apr 2021 18:26:23 +0000",https://www.bostonglobe.com/2021/04/17/sports/...
8385,bostonglobe.com,news,Harry and William were seen chatting together ...,"Sat, 17 Apr 2021 18:25:19 +0000",https://www.bostonglobe.com/2021/04/17/world/h...


In [14]:
# 새로 받아온 것에서 원래 db에 있는 것들을 날림
df_new = pd.merge(df_news_base_data, df_news_url_db, how='outer', indicator='Exist')
df_new = df_new.loc[df_new['Exist']=='left_only']
df_new = df_new.loc[:, ['source','topic','title','publish_date','link']]
df_new = df_new.reset_index(drop=True)
df_new

,source,topic,title,publish_date,link
0,yahoo.com,news,Police: Suspect on the run in fatal shooting o...,2021-04-18T18:15:49Z,https://news.yahoo.com/ems-3-adults-fatally-sh...
1,yahoo.com,news,'Highly unlikely' Chauvin trial ends in 'all-o...,2021-04-18T18:19:00Z,https://news.yahoo.com/highly-unlikely-chauvin...
2,yahoo.com,news,Israeli intel agencies believe Vienna talks wi...,2021-04-18T19:06:19Z,https://news.yahoo.com/israeli-intel-agencies-...
3,yahoo.com,news,DeSantis Hints at Political Consequences for C...,2021-04-18T15:58:42Z,https://news.yahoo.com/desantis-hints-politica...
4,yahoo.com,news,Suspect apprehended in fatal shooting at Wisco...,2021-04-18T10:00:55Z,https://news.yahoo.com/police-3-dead-2-wounded...
...,...,...,...,...,...
1008,macrobusiness.com.au,economics,Daily iron ore price update (supply response),"Sun, 18 Apr 2021 23:20:08 +0000",https://www.macrobusiness.com.au/2021/04/daily...
1009,macrobusiness.com.au,economics,The “pink recession” is actually a “pink boom”,"Sun, 18 Apr 2021 23:00:22 +0000",https://www.macrobusiness.com.au/2021/04/the-p...
1010,macrobusiness.com.au,economics,Robots buy the Australian dollar,"Sun, 18 Apr 2021 14:10:41 +0000",https://www.macrobusiness.com.au/2021/04/robot...
1011,macrobusiness.com.au,economics,Capital Economics: Australian house prices to ...,"Sun, 18 Apr 2021 14:05:22 +0000",https://www.macrobusiness.com.au/2021/04/capit...


In [15]:
con = sqlite3.connect('news_urls.db')
df_new.to_sql('news_urls', con, if_exists='append', index_label='id')
con.close()